# Parte di trave a rastremazione semplice
Capitolo 6.4.2 EC5. Fino a $0.5 h_{ap}$ dall'asse di simmetria

In [4]:
import numpy as np

In [5]:
beamType = 'DoppiaRastremazione'

# beamType = 'Curva'
# beamType = 'Centinata'

timberType = 'lamellare'
# timberType = 'massiccio'

In [90]:
# N/mm2
f_mk = 24
f_vk = 2.7
f_c90k = 2.7
f_t90k = 0.4
E0mean = 11600
E005 = 9400
Gmean = 720

gammaM = 1.45
kmod = 0.9

alpha = 3.15 # in gradi °
b = 220
h_ap = 1250 # altezza apice
h_0 = 700 # altezza appoggio
l = 22000 

alpha = np.radians(alpha)

In [7]:
# volume della trave 
V_b = (h_ap + h_0) * l/2 * b 
V_b = V_b/10**9 # da mm3 a m3
print('V_b:', V_b, 'm3')

# vale solo per la trave a doppia rastremazione:
# volume della zona di colmo
# formula alternativa: b * h_ap**2 * (1 - 0.25 * np.tan(alpha)) / 10**9
h_ap_05 = h_ap - np.tan(alpha)*0.5*h_ap
V_colmo =  (h_ap_05 + h_ap) * 0.5*h_ap * b
V_colmo = V_colmo/10**9
print('V_colmo:', V_colmo, 'm3')

# volume zona di colmo per la normativa
V = min(2/3 * V_b , V_colmo)
print('2/3 V_b:', 2/3 * V_b, 'm3')
print('V:', V, 'm3')

V_0 = 0.01

V_b: 4.719 m3
V_colmo: 0.33902057321078094 m3
2/3 V_b: 3.146 m3
V: 0.33902057321078094 m3


In [8]:
M_d = 551.875 #kNm
V_d = 110.375 #kN

M_d = M_d * 10**6 
V_d = V_d * 10**3 

HP: $ \sigma_{m,\alpha,d} = \sigma_{m,0,d}$

In [9]:
sigma_m_alpha_d = 6 * M_d / (b*h_ap**2) # nel punto in mezzeria
sigma_m_alpha_d

9.632727272727273

# Parte di trave a doppia rastremazione
Capitolo 6.4.3 EC5

## Flessione

In [10]:
f_md = kmod * f_mk / gammaM
f_md

14.896551724137932

da ricontrollare i k

In [11]:
t = 30 # spessore lamelle LVL
r_in = 17000 #raggio interno se curva o centinata. altrimenti non serve
if beamType == 'DoppiaRastremazione':
    k_r = 1
else:
    k_r = 1 if r_in/t >240 else 0.76 + 0.001 * r_in/t
    # t spessore lamelle 
k_r    

1

In [20]:
alpha_ap = np.radians(3.15) # vedi la figura
if beamType == 'DoppiaRastremazione':
    k_l = k_1 = 1 + 1.4*np.tan(alpha_ap) + 5.4*np.tan(alpha_ap)**2
    print('k_l: ', k_l)
else:
    k_1 = 1 + 1.4*np.tan(alpha_ap) + 5.4*np.tan(alpha_ap)**2
    k_2 = 0.35 - 8*np.tan(alpha_ap)
    k_3 = 0.6 + 8.3*np.tan(alpha_ap) - 7.8*np.tan(alpha_ap)**2
    k_4 = 6*np.tan(alpha_ap)**2
    r = r_in + 0.5 * h_ap
    k_l = k_1 + k_2*(h_ap/r) + k_3*(h_ap/r)**2 + k_4*(h_ap/r)**3
    print(k_1)
    print(k_2)
    print(k_3)
    print(k_4)
    print(r)
    print('k_l: ', k_l)

sigma_md = k_l * 6 * M_d / (b*h_ap**2) # nel punto in mezzeria

print('sigma_md: ', sigma_md)
print('f_md: ', f_md)
sigma_md < k_r *f_md


k_l:  1.0934014658411295
sigma_md:  10.532438120047827
f_md:  14.896551724137932


True

## Trazione 90 all'apice

In [13]:
k_dis =  1.4 if beamType == 'DoppiaRastremazione' or  beamType == 'Curva' else  1.7
print('k_dis:', k_dis)

k_vol = 1 if timberType == 'massiccio' else (V_0/V)**0.2 
print('k_vol:', k_vol)

k_dis: 1.4
k_vol: 0.49425923111187087


In [73]:
f_t90d = f_t90k * kmod / gammaM
print('f_t90d:', f_t90d)

f_t90d: 0.24827586206896554


In [15]:
k_dis * k_vol * f_t90d

0.17179769136578132

In [71]:
# solo per trave doppia rastremazione. aggiungere le altre!
k_p = 0.2 * np.tan(alpha) 
print('k_p:', k_p)

k_p: 0.011006665982182707


In [72]:
sigma_t90d = 6 * M_d / (b*h_ap**2) # M all'apice!
print('k_sigma_t90d:', sigma_t90d)

k_sigma_t90d: 9.632727272727273


In [18]:
k_p * sigma_t90d

0.10602421158837087

In [19]:
k_p * sigma_t90d < k_dis * k_vol * f_t90d

True

## Taglio + Trazione a $0.5\cdot h_{ap}$ 

In [43]:
print('x di 0.5*h_ap: ', l/2 -0.5*h_ap) 
print('h trave a 0.5*h_ap: ', h_ap_05)

x di 0.5*h_ap:  10375.0
h trave a 0.5*h_ap:  1215.6041688056791


In [76]:
f_vd = f_vk * kmod / gammaM
f_vd

1.6758620689655175

In [75]:
b_eff = 2.5/f_vk * b
print(b_eff)

tau_d = V * 1.5 / (b_eff * h_ap)
tau_d

203.70370370370367


0.04064727272727273

In [65]:
M = 549.42 *10**6
V = 6.9    *10**3

tau_d_05ap = 1.5*V/(0.67*b*h_ap_05)

In [69]:
def sigma_t90d(M_d,b,h,k_p):
    sigma_t90d = 6 * M_d / (b*h**2) * k_p
    print('sigma_t90d: ', sigma_t90d)
    return sigma_t90d

sigma_t90d_05ap = sigma_t90d(M , b , h_ap_05 , k_p)

sigma_t90d:  0.11161034735739953


In [68]:
tau_d_05ap/f_vd + sigma_t90d_05ap/(f_t90d * k_dis * k_vol) 

0.6841292173801468

In [70]:
tau_d_05ap/f_vd + sigma_t90d_05ap/(f_t90d * k_dis * k_vol) < 1

True

In [ ]:
M_max = 382*10**6 # M corrispondente alla x_max

In [97]:
# Solo se trave simmetrica e carico distribuito!
x_max = l * h_0 / (2*h_ap)
print('x_max: ', x_max)

h_max = h_ap - np.tan(alpha)*(l/2 -x_max) # altezza corrispondente alla x_max
print('h_max: ', h_max)

sigma_md_max = 6*M_max / (b*h_max**2)
print('sigma_md_max: ', sigma_md_max)

x_max:  6160.0
h_max:  983.6386832311784
sigma_md_max:  10.767645103110338


In [93]:
f_c90d = f_c90k * kmod / gammaM
f_c90d

1.6758620689655175

In [95]:
k_mAlpha_compressione = 1 / np.sqrt(1 + (np.tan(alpha)*f_md/(1.5*f_vd))**2 + ((np.tan(alpha))**2*f_md/f_c90d)**2 )
k_mAlpha_compressione

0.9504083645001634

In [96]:
k_mAlpha_compressione * f_md

14.157807360830022

1.6758620689655175

## Stabilità flessotorsionale

In [113]:
# Vedi pagina 291 del libro
h=h_ap

l_t = l # distanza tra due ritegni torsionali successivi 
a1 = 1.13
a2 = 1.44
az = h/2

# caso generico per il RETTANGOLO e non semplificato per h/b > 4
B = E0mean * b**3 * h / 12 # rigidezza flessionale attorno asse z
T = Gmean * b**3 * h / (3*(1 + 0.6 * b/h)) # rigidezza torsionale 

#l_eff = l_t / (a1 * (1 - a2 * az/l_t * np.sqrt(B/T)))
l_eff = 0.9*l

sigma_mcrit = np.pi/l_eff * b**2 / h * E005 * np.sqrt(Gmean/E0mean)
lambda_relm = np.sqrt(f_mk/sigma_mcrit)
if lambda_relm < 0.75:
    k_crit = 1
elif lambda_relm < 1.14:
    k_crit = 1.56 - .75*lambda_relm
else:
    k_crit = 1/(lambda_relm**2)

print('l_eff: ', l_eff)
print('l_eff/l: ', l_eff/l)
print('sigma_mcrit: ', sigma_mcrit)
print('lambda_relm: ', lambda_relm)
print('k_crit: ', k_crit)

l_eff:  19800.0
l_eff/l:  0.9
sigma_mcrit:  14.387493428411121
lambda_relm:  1.2915554362115353
k_crit:  0.5994788928504635


In [121]:
T

2889290882778.582

In [118]:
B

12866333333333.334

In [119]:
J

NameError: name 'J' is not defined

In [110]:
sigma_md < k_crit * f_md

False

In [109]:
sigma_md

10.532438120047827

In [111]:
k_crit * f_md

8.29795207757759